# Hyperparameter Tuning

Scikit-learn documentation:
* [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)
* [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html)
* [Breast Cancer dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html)

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/murilogustineli/hype-tuning/blob/main/gridsearch.ipynb)

### Import libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
np.random.seed(42)

## Part 1: Machine Learning intro

### Load data from sklearn

In [ ]:
from sklearn.datasets import load_breast_cancer

# Load Breast Cancer dataset from sklearn
X, y = load_breast_cancer(return_X_y=True, as_frame=True)

# Data dimensions
X.shape, y.shape

In [ ]:
# Distribution of target variable
y.value_counts()

Dataset summary:
1. The Breast Cancer dataset has 569 instances and 31 features
    * Each instance has a binary target variable indicating the patient’s diagnosis (malignant or benign)
    * `Malignant` == 0
    * `Benign` == 1
2. The target class distribution is **imbalanced!**
    * 357 instances diagnosed as benign and 212 as malignant.

### Training a model

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score

# KNN model
clf = KNeighborsClassifier()


# Fit model
clf.fit(X, y)
# Make predictions
y_pred = clf.predict(X)
# Scores
score = clf.score(X, y)
f1 = f1_score(y_pred, y)

# Scores
print(f"{clf.__class__.__name__}")
print(f"Accuracy: {round(score, 3)}")
print(f"F1 Score: {round(f1, 3)}")

In [ ]:
# KNN object
clf

In [ ]:
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier

# DecisionTree
clf = DecisionTreeClassifier(max_depth=2)

# Fit model
clf.fit(X, y)
# Make predictions
y_pred = clf.predict(X)
# Scores
score = clf.score(X, y)
f1 = f1_score(y_pred, y)

# Scores
print(f"{clf.__class__.__name__}")
print(f"Accuracy: {round(score, 3)}")
print(f"F1 Score: {round(f1, 3)}")

In [ ]:
# Visualize Decision Tree
feature_names = list(X.columns)
class_names = ['Malignant', 'Benign'] # Malignant==0, Benign==1

# Init plot
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(4, 4), dpi=200)
tree.plot_tree(
    clf,
    feature_names=feature_names,
    class_names=class_names,
    filled=True,
    fontsize=5,
    rounded=True);
fig.tight_layout(pad=3);

## Part 2: Pipelines

### Scaling features
`Scaling` features is important because it can help improve the performance of the model.
1. Scaling ensures that each feature is on a similar scale.
2. Prevents some features from dominating others in terms of their influence on teh model.
3. Help convergence of certain algorithms.
4. Make the model more robust to outliers and noise in the data.

In [ ]:
# Rename columns to lower case
cols = list(X.columns)
lower_cols = [col.replace(" ", "_").lower() for col in cols]
X.columns = lower_cols

In [ ]:
# Get max from each feature
mean_radius_max = np.max(X['mean_area'])
mean_area_max = np.max(X['mean_smoothness'])

print(f'Max mean area:   {mean_radius_max}')
print(f'Max mean smooth: {mean_area_max}')

In [ ]:
# Plot different features
def plot_scales(df, feature1, feature2):
    fig, ax = plt.subplots(figsize=(6, 4), dpi=150)
    ax.scatter(df[feature1], df[feature2])
    ax.set_title('Different scales between two features')
    ax.set_xlabel(f"{feature1.replace('_', ' ').title()}")
    ax.set_ylabel(f"{feature2.replace('_', ' ').title()}")
    ax.grid(color='blue', linestyle='--', linewidth=1, alpha=0.2)
    for spine in ['top', 'right']:
      ax.spines[spine].set_visible(False)
    fig.tight_layout(pad=3)

In [ ]:
# Plot difference
plot_scales(df=X, feature1='mean_area', feature2='mean_smoothness')

### StandardScaler

In [ ]:
from sklearn.preprocessing import StandardScaler

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

In [ ]:
# Plot difference
plot_scales(df=X_scaled, feature1='mean_area', feature2='mean_smoothness')

### We have to rethink what our model actually is!

### Creating your first pipeline
Let's put everything into a pipeline!

### KNN pipeline

In [ ]:
from sklearn.pipeline import Pipeline

# KNN pipeline
pipe = Pipeline(steps=[
    ('scale', StandardScaler()),
    ('model', KNeighborsClassifier())])

# Fit model
pipe.fit(X, y)
# Make predictions
y_pred = pipe.predict(X)
# Scores
score = pipe.score(X, y)
f1 = f1_score(y_pred, y)

# Scores
print(f"{pipe['model'].__class__.__name__}")
print(f"Accuracy: {round(score, 3)}")
print(f"F1 Score: {round(f1, 3)}")

In [ ]:
# Pipeline object
pipe

In [ ]:
# Pipeline scaler
pipe['scale']

In [ ]:
# Pipeline model
pipe['model']

### DecisionTree pipeline

In [ ]:
# Decision Tree pipeline
pipe = Pipeline(steps=[
    ('scale', StandardScaler()),
    ('model', DecisionTreeClassifier(max_depth=2))])

# Fit model
pipe.fit(X, y)
# Make predictions
y_pred = pipe.predict(X)
# Scores
score = pipe.score(X, y)
f1 = f1_score(y_pred, y)

print(f"Accuracy: {round(score, 3)}")
print(f"F1 Score: {round(f1, 3)}")

In [ ]:
# Pipeline object
pipe

### Run different pipelines

In [ ]:
# Function to run pipelines
def fit_predict(pipe: object, X: pd.DataFrame,  y: pd.DataFrame):
    # Fit model
    pipe.fit(X, y)
    # Make predictions
    y_pred = pipe.predict(X)
    # Scores
    score = pipe.score(X, y)
    f1 = f1_score(y_pred, y)
    
    print(f"{str(pipe['model'].__class__.__name__)}")
    print(f"Accuracy: {round(score, 3)}")
    print(f"F1 Score: {round(f1, 3)}\n")

In [ ]:
# KNN pipeline
knn_pipe = Pipeline(steps=[
    ('scale', StandardScaler()),
    ('model', KNeighborsClassifier(n_neighbors=5))])

# Decision Tree pipeline
dt_pipe = Pipeline(steps=[
    ('scale', StandardScaler()),
    ('model', DecisionTreeClassifier(max_depth=2))])

# Fit pipelines
fit_predict(pipe=knn_pipe, X=X, y=y)
fit_predict(pipe=dt_pipe, X=X, y=y)

### We're making a HUGE mistake!!!
How do we know we are able to **generalize** to new data?
* We used the entire data for training and testing, and that's bad!!
* We don't want to evaluate model performance on the same dataset we used to train it.
    * **Generalization** is about the ability of a model to perform well on unseen data.
* We need to split the data intro `training` and `testing`.
    * Use the `training` data to fit the model and make predictions
    * Use the `testing` data to test the model performance

### Train/Test split

In [ ]:
from sklearn.model_selection import train_test_split

# Train/Test Split using Stratified Sampling
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42)

print(f"Train data: {X_train.shape, y_train.shape}")
print(f"Test data:  {X_test.shape, y_test.shape}")

### Handle imbalanced data
The data is also imbalanced, where one class have more instances than another class.
* 357 instances diagnosed as `benign` and 212 as `malignant`.

In [ ]:
from imblearn.over_sampling import SMOTE

# SMOTE (Synthetic Minority Oversampling Technique)
sm = SMOTE(random_state=42)
X_train, y_train = sm.fit_resample(X_train, y_train)

print(f"Train data: {X_train.shape, y_train.shape}")
print(f"Test data:  {X_test.shape, y_test.shape}")

In [ ]:
# Class distribution after SMOTE
y_train.value_counts()

In [ ]:
# Test data class distribution
y_test.value_counts()

### Preprocess data
Putting all the steps above together

In [ ]:
# Preprocess data
def preprocess_data(test_size=0.2, oversample=False) -> tuple:
    # Load dataset
    X, y = load_breast_cancer(return_X_y=True, as_frame=True)

    # Train/Test Split using Stratified Sampling
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, stratify=y, random_state=42)

    # Oversampling using SMOTE
    if oversample:
        sm = SMOTE(random_state=42)
        X_train, y_train = sm.fit_resample(X_train, y_train)

    return X_train, X_test, y_train, y_test

In [ ]:
# Get data
X_train, X_test, y_train, y_test = preprocess_data(test_size=0.2, oversample=True)

print(f"Train data: {X_train.shape, y_train.shape}")
print(f"Test data:  {X_test.shape, y_test.shape}")

In [ ]:
# KNN pipeline
knn_pipe = Pipeline(steps=[
    ('scale', StandardScaler()),
    ('model', KNeighborsClassifier())])

# Decision Tree pipeline
dt_pipe = Pipeline(steps=[
    ('scale', StandardScaler()),
    ('model', DecisionTreeClassifier(max_depth=1))])

# Fit pipelines
fit_predict(pipe=knn_pipe, X=X, y=y)
fit_predict(pipe=dt_pipe, X=X, y=y)

## Part 3: GridSearch

* `GridSearch` is a technique used to find the optimal hyperparameters for a machine learning model.
* `Hyperparameters` are certain values or weights that determine the learning process of an algorithm..
* `Hyperparameter tuning` is the process of finding the best hyperparameters for a given machine learning algorithm and dataset.
    * The performance of a machine learning model is highly dependent on the values of its hyperparameters.
    * Choosing the right hyperparameters is critical for achieving good performance.
* `Stratified Sampling` ensures the population is divided into homogeneous subgroups where the right amount of instances is sampled from each class
    * Guarantees that the test set is representative of the overall population

**Scikit-learn documentation:**
* [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)
* [`DecisionTreeClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)
* [`KNeighborsClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)

In [ ]:
# Looking at DecisionTree parameters
dt_pipe['model'].get_params()

### GridSearch for DecisionTree

In [ ]:
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold

# Decision Tree pipeline
dt_pipe = Pipeline(steps=[
    ('scale', StandardScaler()),
    ('model', DecisionTreeClassifier())])

# DecisionTree GridSearchCV params
dt_param_grid = {
    'model__criterion': ['gini', 'entropy'],
    'model__max_depth': list(range(1, 11, 1)),
    'model__min_samples_leaf': list(range(1, 11, 1))}

# Stratified sampling
strat_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Init GridSearchCV
gridsearch = GridSearchCV(
    estimator=dt_pipe,
    param_grid=dt_param_grid,
    scoring='f1',
    cv=strat_kfold,
    verbose=2,
    n_jobs=-1)

In [ ]:
# Fit model
dt_clf = gridsearch.fit(X_train, y_train)

#### Why 200 candidates? Why 1,000 fits?

In [ ]:
criterion = ['gini', 'entropy']
max_depth = list(range(1, 11, 1))
min_samples_leaf = list(range(1, 11, 1))
n_splits = 5

candidates = len(criterion) * len(max_depth) * len(min_samples_leaf)
print(f'Candidates: {candidates}')
print(f'Total fits: {candidates*n_splits}')

In [ ]:
# Look at GridSearch object
dt_clf

In [ ]:
# Best estimator
dt_clf.best_estimator_

In [ ]:
# Best parameters
dt_clf.best_params_

### Make predictions

In [ ]:
# Train and test scores
train_score = dt_clf.score(X_train, y_train)
test_score = dt_clf.score(X_test, y_test)

print(f'Train score: {round(train_score, 3)}')
print(f'Test score:  {round(test_score, 3)}')

In [ ]:
# F1 score
y_pred = dt_clf.predict(X_test)
f1 = f1_score(y_test, y_pred)
print(f'F1 score: {round(f1, 3)}')

### Performance metrics

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

# Display scores
model_name = dt_clf.best_estimator_['model'].__class__.__name__
print(f"{dt_clf.__class__.__name__}:\t {model_name}")
print(f"Train score:     {round(train_score, 3)}")
print(f"Test score:      {round(test_score, 3)}")
print(f"Accuracy score:  {round(accuracy, 3)}")
print(f"Precision score: {round(precision, 3)}")
print(f"Recall score:    {round(recall, 3)}")
print(f"F1 score:        {round(f1, 3)}")
print(f"ROC AUC score:   {round(roc_auc, 3)}")

In [ ]:
# Look at GridSearchCV results
df = pd.DataFrame(gridsearch.cv_results_)
df.head(15)

### GridSearch for KNN

In [ ]:
# KNN pipeline
knn_pipe = Pipeline(steps=[
    ('scale', StandardScaler()),
    ('model', KNeighborsClassifier())])

# KNN GridSearchCV params
knn_param_grid = {
    'model__n_neighbors': list(range(5, 55, 5)),
    'model__weights' : ['uniform', 'distance'],
    'model__metric': ['euclidean', 'manhattan']}

# Stratified sampling
strat_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Init GridSearchCV
gridsearch = GridSearchCV(
    estimator=knn_pipe,
    param_grid=knn_param_grid,
    scoring='f1',
    cv=strat_kfold,
    verbose=2,
    n_jobs=-1)

In [ ]:
# Fit model
knn_clf = gridsearch.fit(X_train, y_train)

In [ ]:
knn_clf.best_estimator_

In [ ]:
# Metrics
train_score = dt_clf.score(X_train, y_train)
test_score = dt_clf.score(X_test, y_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

# Display scores
model_name = knn_clf.best_estimator_['model'].__class__.__name__
print(f"{knn_clf.__class__.__name__}:\t {model_name}")
print(f"Train score:     {round(train_score, 3)}")
print(f"Test score:      {round(test_score, 3)}")
print(f"Accuracy score:  {round(accuracy, 3)}")
print(f"Precision score: {round(precision, 3)}")
print(f"Recall score:    {round(recall, 3)}")
print(f"F1 score:        {round(f1, 3)}")
print(f"ROC AUC score:   {round(roc_auc, 3)}")

## Part 4: Going deeper

> _"Implementing machine learning is first and foremost a software endeavour, and requires experience building well architected, reliable, easy to deploy software."_

### Learner Class
Each model has its own pipeline and gridsearch parameters.

**Documentation:**
* [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)
* [`RandomizedSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html)
* [`DecisionTreeClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)
* [`KNeighborsClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)
* [`XGBClassifier`](https://xgboost.readthedocs.io/en/stable/parameter.html)
* [`MLPClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html)

In [ ]:
import time
from tqdm import tqdm
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, classification_report

In [ ]:
# Learner class
class Learner:
    def __init__(self, pipe, params):
        self.pipe = pipe
        self.params = params
        self.clf = None
        self.scores = None
        self.search_name = None
        self.class_report = None
        self.dataset_name = None
        self.learning_curve = {}
        self.validation_curve = {}
        self.cv = StratifiedKFold(n_splits=5, shuffle=True)
        self.name = str(self.pipe['model'].__class__.__name__)
        
    def fit_gridsearch(self, search_func, X_train, y_train, verbose=False):
        np.random.seed(42)

        # Train learner
        self.clf = search_func(
            self.pipe,
            self.params,
            scoring={
                'accuracy': make_scorer(accuracy_score),
                'precision': make_scorer(precision_score),
                'recall': make_scorer(recall_score),
                'f1': make_scorer(f1_score),
                'roc_auc': make_scorer(roc_auc_score)},
            refit='f1',
            cv=self.cv,
            verbose=verbose,
            n_jobs=-1)
        # Fit the model
        self.clf.fit(X_train, y_train)
        self.search_name = str(self.clf.__class__.__name__)

    def get_scores(self, X_train, X_test, y_train, y_test):
        if self.search_name == 'Benchmark':
            best_estimator = self.clf
        else:
            best_estimator = self.clf.best_estimator_
        
        np.random.seed(42)
        # Score on training data
        start_time = time.time()
        best_estimator.fit(X_train ,y_train)
        end_time = time.time()
        wall_clock_fit = end_time - start_time
        # train_score = self.clf.score(X_train, y_train)
        train_score = best_estimator.score(X_train, y_train)

        # Score on test data
        start_time = time.time()
        # y_pred = self.clf.predict(X_test)
        y_pred = best_estimator.predict(X_test)
        end_time = time.time()
        wall_clock_pred = end_time - start_time
        # test_score = self.clf.score(X_test, y_test)
        test_score = best_estimator.score(X_test, y_test)
        # Metrics
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        roc_auc = roc_auc_score(y_test, y_pred)
        # Classification report
        self.class_report = classification_report(y_test, y_pred)

        self.scores = {
            'train_score': round(train_score, 3),
            'test_score': round(test_score, 3),
            'accuracy': round(accuracy, 3),
            'precision': round(precision, 3),
            'recall': round(recall, 3),
            'f1': round(f1, 3),
            'roc_auc': round(roc_auc, 3),
            'wall_clock_fit': wall_clock_fit,
            'wall_clock_pred': wall_clock_pred}

    # Evaluate Learner class
    def evaluate_learner(self):
        print(f"{'#################################'*2}")
        print(f'{self.search_name}:\t  {self.name}')
        print(f"Train score:     {round(self.scores['train_score'], 3)}")
        print(f"Test score:      {round(self.scores['test_score'], 3)}")
        print(f"Accuracy score:  {round(self.scores['accuracy'], 3)}")
        print(f"Precision score: {round(self.scores['precision'], 3)}")
        print(f"Recall score:    {round(self.scores['recall'], 3)}")
        print(f"F1 score:        {round(self.scores['f1'], 3)}")
        print(f"ROC AUC score:   {round(self.scores['roc_auc'], 3)}")
        print(f"Wall Clock Fit:  {round(self.scores['wall_clock_fit'], 3)}")
        print(f"Wall Clock Pred: {round(self.scores['wall_clock_pred'], 3)}")
        # Classification report
        print(f"\nClassification report:\n{self.class_report}")
        
        # Best score and best params
        print(f"Best score: {round(self.clf.best_score_, 3)}")
        print("Best params:")
        for param in self.clf.best_params_.items():
            print(f"\t{param}")
        print()

### Pipe setup

In [ ]:
def learner_setup():
    # KNN pipeline
    knn_pipe = Pipeline(steps=[
        ('scaler', StandardScaler()),
        ('model', KNeighborsClassifier())])
    # GridSearchCV params
    knn_param_grid = {
        'model__n_neighbors': list(range(1, 9, 1)),
        'model__weights' : ['uniform', 'distance'],
        'model__metric': ['euclidean', 'manhattan']}


    # Decision Tree pipeline
    dt_pipe = Pipeline(steps=[
        ('scaler', StandardScaler()),
        ('model', DecisionTreeClassifier(random_state=42))])
    # GridSearchCV params
    dt_param_grid = {
        'model__criterion': ['gini', 'entropy'],
        'model__max_depth': range(1, 8, 1),
        'model__min_samples_leaf': range(1, 6, 1)}


    # XGBoost pipeline
    xgb_pipe = Pipeline(steps=[
        ('scaler', StandardScaler()),
        ('model', XGBClassifier(seed=42))])
    # GridSearchCV params
    xgb_param_grid = {
        'model__objective': ['binary:logistic'],
        'model__n_estimators': np.arange(100, 300, 50),
        'model__learning_rate': np.arange(0.1, 0.2, 0.05),
        'model__max_depth': [3, 4, 5, 6],
        'model__min_child_weight': np.arange(1, 5, 1),
        'model__subsample': np.arange(0.6, 1.0, 0.1),
        'model__colsample_bytree': np.arange(0.6, 1.0, 0.1)}

    # MLP pipeline
    mlp_pipe = Pipeline(steps=[
        ('scaler', StandardScaler()),
        ('model', MLPClassifier(random_state=42))])
    # GridSearchCV params
    mlp_param_grid = {
        'model__activation': ['relu', 'tanh', 'logistic'],
        'model__max_iter': [1000],
        'model__hidden_layer_sizes': [(2,),(4,),(8,),(16,),(32,),(64,)],
        'model__learning_rate': ['constant', 'adaptive'],
        'model__learning_rate_init': [0.001, 0.01, 0.1, 1]}


    # Init learners
    knn_grid = Learner(pipe=knn_pipe, params=knn_param_grid)
    dt_grid = Learner(pipe=dt_pipe, params=dt_param_grid)
    xgb_grid = Learner(pipe=xgb_pipe, params=xgb_param_grid)
    mlp_grid = Learner(pipe=mlp_pipe, params=mlp_param_grid)

    # List of learners
    learners = [
        knn_grid,
        dt_grid,
        xgb_grid,
        mlp_grid
    ]

    return learners

### Train Learners using `RandomizedSearchCV`

In [ ]:
# Setup learners
learners = learner_setup()

# Fit learners
with tqdm(learners, unit='batch') as tepoch:
    for learner in tepoch:
        tepoch.set_description("Training progress")   
        # Fit GridSearchCV and get scores
        learner.fit_gridsearch(RandomizedSearchCV, X_train, y_train)
        learner.get_scores(X_train, X_test, y_train, y_test)

### Results

In [ ]:
# Evaluate learner performance
for learner in learners:
    learner.evaluate_learner()

In [ ]:
# Trained learners
knn, dt, xgb, mlp = learners

In [ ]:
# KNN best estimator
knn.clf.best_estimator_

In [ ]:
# DecisionTree best estimator
dt.clf.best_estimator_

In [ ]:
# XGBoost best estimator
xgb.clf.best_estimator_

In [ ]:
# MLPClassifier best estimator
mlp.clf.best_estimator_

### Comparing Performance

In [ ]:
# Function to run pipelines
def fit_predict(pipe, X_train, X_test, y_train, y_test):
    # Fit model
    pipe.fit(X_train, y_train)
    # Make predictions
    y_pred = pipe.predict(X_test)
    # Scores
    score = pipe.score(X_test, y_test)
    f1 = f1_score(y_pred, y_test)
    
    # Print results
    print(f"{str(pipe['model'].__class__.__name__)}")
    print(f"Accuracy: {round(score, 3)}")
    print(f"F1 Score: {round(f1, 3)}\n")

    return round(f1, 3)

### Without tuning

In [ ]:
# KNN pipeline
knn_pipe = Pipeline(steps=[
    ('scale', StandardScaler()),
    ('model', KNeighborsClassifier(n_neighbors=20))])

# Decision Tree pipeline
dt_pipe = Pipeline(steps=[
    ('scale', StandardScaler()),
    ('model', DecisionTreeClassifier(max_depth=1, random_state=42))])

# XGBoost pipeline
xgb_pipe = Pipeline(steps=[
    ('scale', StandardScaler()),
    ('model', XGBClassifier(max_depth=1, seed=42))])

# Pipe setup
mlp_pipe = Pipeline(steps=[
    ('scale', StandardScaler()),
    ('model', MLPClassifier(
        hidden_layer_sizes=(10,10,), max_iter=1000, random_state=42))])

# Fit pipelines
knn_f1 = fit_predict(knn_pipe, X_train, X_test, y_train, y_test)
dt_f1 = fit_predict(dt_pipe, X_train, X_test, y_train, y_test)
xgb_f1 = fit_predict(xgb_pipe, X_train, X_test, y_train, y_test)
mlp_f1 = fit_predict(mlp_pipe, X_train, X_test, y_train, y_test)

### Tuned hyperparameters

In [ ]:
def tuned_setup():
    # KNN pipeline
    knn_pipe = Pipeline(steps=[
        ('scaler', StandardScaler()),
        ('model', KNeighborsClassifier())])
        
    knn_tuned_params = {
        'model__weights': ['distance'],
		'model__n_neighbors': [7],
		'model__metric': ['manhattan']}

    # Decision Tree pipeline
    dt_pipe = Pipeline(steps=[
        ('scaler', StandardScaler()),
        ('model', DecisionTreeClassifier(random_state=42))])
    
    dt_tuned_params = {
		'model__min_samples_leaf': [3],
		'model__max_depth': [5],
		'model__criterion': ['gini']}
    
	# XGBoost pipeline
    xgb_pipe = Pipeline(steps=[
        ('scaler', StandardScaler()),
        ('model', XGBClassifier(seed=42))])
        
    xgb_tuned_params = {
		'model__subsample': [0.6],
		'model__objective': ['binary:logistic'],
		'model__n_estimators': [250],
		'model__min_child_weight': [2],
		'model__max_depth': [4],
		'model__learning_rate': [0.15],
		'model__colsample_bytree': [0.7]}

    # MLP pipeline
    mlp_pipe = Pipeline(steps=[
        ('scaler', StandardScaler()),
        ('model', MLPClassifier(random_state=42))])
        
    mlp_tuned_params = {
		'model__max_iter': [1000],
		'model__learning_rate_init': [0.001],
		'model__learning_rate': ['constant'],
		'model__hidden_layer_sizes': [(3,)],
		'model__activation': ['relu']}

    # Init learners
    knn_grid = Learner(pipe=knn_pipe, params=knn_tuned_params)
    dt_grid = Learner(pipe=dt_pipe, params=dt_tuned_params)
    xgb_grid = Learner(pipe=xgb_pipe, params=xgb_tuned_params)
    mlp_grid = Learner(pipe=mlp_pipe, params=mlp_tuned_params)

    # List of learners
    learners = [
        knn_grid,
        dt_grid,
        xgb_grid,
        mlp_grid
    ]

    return learners

### Train tuned learners

In [ ]:
# Setup learners
learners = tuned_setup()

# Fit learners
with tqdm(learners, unit='batch') as tepoch:
    for learner in tepoch:
        tepoch.set_description("Training progress")   
        # Fit GridSearchCV and get scores
        learner.fit_gridsearch(GridSearchCV, X_train, y_train)
        learner.get_scores(X_train, X_test, y_train, y_test)

# Trained learners
knn, dt, xgb, mlp = learners

In [ ]:
# KNN GridSearchCV results
print(knn.name)
print(f"Accuracy: {knn.scores['accuracy']}")
print(f"F1 Score: {knn.scores['f1']}\n")

# DecisionTree GridSearchCV results
print(dt.name)
print(f"Accuracy: {dt.scores['accuracy']}")
print(f"F1 Score: {dt.scores['f1']}\n")

# XGBoost GridSearchCV results
print(xgb.name)
print(f"Accuracy: {xgb.scores['accuracy']}")
print(f"F1 Score: {xgb.scores['f1']}\n")

# MLP GridSearchCV results
print(mlp.name)
print(f"Accuracy: {mlp.scores['accuracy']}")
print(f"F1 Score: {mlp.scores['f1']}\n")

### Performance Comparison

In [ ]:
# Plot final results
def plot_model_results(no_search, grid_search):
    fig, ax = plt.subplots(figsize=(6.4, 4.8), dpi=200)
    ax.margins(x=0.1, y=0.1)  # No margins on x and y-axis
    ymin = np.min((no_search, grid_search))
    # Learner names
    short_names = ['KNeighbors', 'DecisionTree', 'XGBoost', 'MLPClassifier']
    models = ['KNN', 'DecisionTree', 'XGBoost', 'MLPClassifier']
    for i in range(len(models)):
        x = np.arange(len(models))
        width = 0.35
        ax.bar(x[i] - width/2, no_search[i], width=width, color='tab:blue')
        ax.bar(x[i] + width/2, grid_search[i], width=width, color='tab:orange')
        ax.set_xticks(x)
        ax.set_xticklabels(short_names)
        ax.annotate(f'{no_search[i]}', xy=(x[i] - width/2, no_search[i]),
                xytext=(0, 3), textcoords="offset points", ha='center', va='bottom')
        ax.annotate(f'{grid_search[i]}', xy=(x[i] + width/2, grid_search[i]),
                xytext=(0, 3), textcoords="offset points", ha='center', va='bottom')
    ax.set_title(f'Overall Performance', weight='bold', fontsize=16)
    ax.set_xlabel('Learning Algorithms')
    ax.set_ylabel('Test Set F1 Score')
    ax.grid(color='blue', linestyle='--', linewidth=1, alpha=0.2)
    ax.legend(['Without Tuning', 'GridSearch'], loc='upper left', fontsize=8)
    for spine in ['top', 'right', 'bottom', 'left']:
        ax.spines[spine].set_visible(False)
    ax.set_ylim([ymin-10, 100])
    fig.tight_layout()
    # plt.savefig(f'./plots/PerformanceSummary.png')
    plt.show();

In [ ]:
# Without tuning
no_search = np.round(np.multiply([knn_f1, dt_f1, xgb_f1, mlp_f1], 100), 3)
# GridSearchCV
grid_search = np.round(np.multiply([
    knn.scores['f1'],
    dt.scores['f1'],
    xgb.scores['f1'],
    mlp.scores['f1']], 100), 3)

# Plot results
plot_model_results(no_search, grid_search)